# VHR Composite Metrics Visualization

Purpose: Visualize the mode outputs for each grid cell in a composite of vhr model outputs.

Installation requirements:

```bash
pip install localtileserver
```


In [1]:
# !pip install localtileserver

In [2]:
import os
import re
import json
import folium
import numpy as np
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

from pprint import pprint

from folium import plugins
from localtileserver import TileClient, get_leaflet_tile_layer
from ipyleaflet import Map, basemaps, ScaleControl, LayersControl
from ipyleaflet import LegendControl, FullScreenControl

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

import localtileserver
from localtileserver import get_folium_tile_layer, TileClient

## Notebook Parameters

- **GRID_BASE_PATH**: str = Basepath where grid zarr files are located
- **TILE**: str = tile to visualize mode product for
- **EXPERIMENT**: str = version and epoch 

In [3]:
GRID_BASE_PATH: str = '/explore/nobackup/people/cssprad1/projects/3sl/3.0.0/data'
TILE: str = 'h24v38'
EXPERIMENT: str = 'v3.2016'

In [4]:
grid_path = os.path.join(GRID_BASE_PATH, EXPERIMENT)

In [5]:
# TODO:
tiles_basemap: str = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
cmap: list = ['#003f5c','#374c80','#7a5195','#bc5090','#ef5675','#ff764a','#ffa600']

In [6]:
legend_dict = {
    'Mode - 1 label': '#003f5c',
    'Mode - 2 label': '#374c80',
    'Mode - 3 label': '#bc5090',
    'Mode - 0,1 label': '#7a5195',
    'Mode - 0,2 label': '#ef5675',
    'Mode - 1,2 label': '#ff764a',
    'Mode - 0,1,2 label': '#ffa600',
}

legend = LegendControl(legend_dict)

In [7]:
# Client - initial client to localize zoom
data_client = TileClient(os.path.join(grid_path, f'CAS.M1BS.{TILE}.mode.{EXPERIMENT}.test.tif'))

# Create ipyleaflet TileLayer from that server
data_layer = get_leaflet_tile_layer(
    data_client, 
    show=False, 
    cmap=cmap,
    nodata=10,
    max_zoom=20,
    name="3sl composite label - mode")

# Create ipyleaflet map, add tile layer, and display
m = Map(
    center=data_client.center(),
    zoom=data_client.default_zoom,
    basemap=basemaps.Esri.WorldImagery,
    scroll_wheel_zoom=True,
    keyboard=True
)
m.add_layer(data_layer)
m.add_control(legend)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

display(m)

Map(center=[13.358142643935759, -15.258837022681705], controls=(ZoomControl(options=['position', 'zoom_in_text…